In [ ]:
import os
import glob
import pandas as pd 
import plotly.express as px

# Get current working directory (should be dash_app)
base_dir = os.getcwd()

# Step up one level to Driver_side_alpha
driver_side_alpha_dir = os.path.dirname(base_dir)

# Construct the path to the modeled folder
folder_path_events = os.path.join(driver_side_alpha_dir, "LA", "Events", "events_data", "LA_events", "modeled")
folder_path_flights = os.path.join(driver_side_alpha_dir, "LA", "flights", "flight_data", "modeled")


# Find matching CSV files
csv_file_events = glob.glob(os.path.join(folder_path_events, "event_clustering*.csv"))
csv_file_flight = glob.glob(os.path.join(folder_path_flights,"Top_clusters*.csv"))

# Load the latest one if available
if csv_file_events:
    latest_events = max(csv_file_events, key=os.path.getmtime)
    df_events = pd.read_csv(latest_events)
else:
    raise FileNotFoundError("No event_clustering CSV files found in the modeled directory.")

if csv_file_flight: 
    latest_flights = max(csv_file_flight,key=os.path.getmtime)
    df_flights = pd.read_csv(latest_flights)
else:
    raise FileNotFoundError("Not found")

In [12]:
df_flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   flight_date       42 non-null     object 
 1   cluster_id        42 non-null     int64  
 2   arrival_iataCode  42 non-null     object 
 3   start_min         42 non-null     int64  
 4   end_min           42 non-null     int64  
 5   flight_count      42 non-null     int64  
 6   start_time        42 non-null     object 
 7   end_time          42 non-null     object 
 8   duration_min      42 non-null     int64  
 9   midpoint_min      42 non-null     int64  
 10  midpoint_time     42 non-null     object 
 11  rank              42 non-null     float64
 12  source            42 non-null     object 
dtypes: float64(1), int64(6), object(6)
memory usage: 4.4+ KB


In [15]:
df_flights.head()

,flight_date,cluster_id,arrival_iataCode,start_min,end_min,flight_count,start_time,end_time,duration_min,midpoint_min,midpoint_time,rank,source
0,2025-07-30,0,mci,925,1062,46,2025-08-04 15:25:00,2025-08-04 17:42:00,137,993,16:33,2.0,flight_90min
1,2025-07-30,1,mci,1291,1439,48,2025-08-04 21:31:00,2025-08-04 23:59:00,148,1365,22:45,1.0,flight_90min
2,2025-07-30,4,mci,1095,1275,36,2025-08-04 18:15:00,2025-08-04 21:15:00,180,1185,19:45,3.0,flight_90min
3,2025-07-31,2,mci,815,995,50,2025-08-04 13:35:00,2025-08-04 16:35:00,180,905,15:05,2.0,flight_90min
4,2025-07-31,3,mci,1360,1435,43,2025-08-04 22:40:00,2025-08-04 23:55:00,75,1397,23:17,3.0,flight_90min


In [ ]:


# Example structure — replace with your actual df_events
# df_events = pd.read_csv("your_file.csv")
# Make sure start_time and end_time are datetime objects
df_flights['start_time'] = pd.to_datetime(df_flights['start_time'])
df_flights['end_time'] = pd.to_datetime(df_flights['end_time'])
df_flights["tooltip"] = "Flights: " + df_flights["flight_count"].astype(str)

flight_90 = df_flights[df_flights['source']=='flight_90min']
flight_180 = df_flights[df_flights['source']!='flight_90min']

# Create timeline plot
timeline_90 = px.timeline(
    flight_90,
    x_start="start_time",
    x_end="end_time",
    color="rank",
    y="flight_date",  # or use another label column
    hover_data="tooltip",
   # color="event_type",  # optional: color by category
    title="📅 LA flight Timeline"
    subtitle=
)

# Reverse Y-axis so earliest events are at the top
timeline_90.update_yaxes(autorange="reversed")

# Show the plot
timeline_90.show()

In [28]:
timeline_180 = px.timeline(
    flight_180,
    x_start="start_time",
    x_end="end_time",
    color="rank",
    hover_data="tooltip",
    y="flight_date",  # or use another label column
   # color="event_type",  # optional: color by category
    title="📅 LA flight Timeline",
    subtitle="3 hour cluster's"
)

# Reverse Y-axis so earliest events are at the top
timeline_180.update_yaxes(autorange="reversed")

# Show the plot
timeline_180.show()

In [29]:
df_events.head()

,id,short_title,popularity,score,type,name,postal_code,address,latitude,longitude,pacific_endtime,date_pacific,start_time_pacific,end_time_pacific,Day_of_week,lat_rad,lon_rad,time_min,time_scaled,cluster
0,17345398,Hollywood Bowl Jazz Festival,0.737866,0.447126,music_festival,Hollywood Bowl,90068.0,2301 Highland Avenue,34.1122,-118.339,2025-08-06 21:00:00-07:00,2025-08-06,08:00 PM,09:00 PM,Wednesday,0.595370,-2.065405,90.0,2.25,0
1,17449955,Primus with Ty Segall,0.760271,0.487892,concert,The Greek Theatre - Los Angeles,90027.0,2700 North Vermont Avenue,34.1205,-118.297,2025-08-06 21:30:00-07:00,2025-08-06,08:00 PM,09:30 PM,Wednesday,0.595515,-2.064672,120.0,3.00,-1
2,6414111,Some Like it Hot - Los Angeles,0.672556,0.323583,broadway,Hollywood Pantages Theatre,90028.0,6233 Hollywood Blvd.,34.1017,-118.325,2025-08-06 19:30:00-07:00,2025-08-06,07:30 PM,07:30 PM,Wednesday,0.595187,-2.065161,0.0,0.00,0
3,17521315,Olivia Dean,0.796990,0.527146,concert,The Fonda Theatre,90028.0,6126 Hollywood Blvd.,34.1014,-118.323,2025-08-06 22:30:00-07:00,2025-08-06,09:00 PM,10:30 PM,Wednesday,0.595182,-2.065126,180.0,4.50,0
4,17506107,CA7RIEL / PACO AMOROSO,0.754110,0.460476,concert,The Novo,90015.0,800 W Olympic Blvd,34.0446,-118.265,2025-08-06 21:30:00-07:00,2025-08-06,08:00 PM,09:30 PM,Wednesday,0.594190,-2.064114,120.0,3.00,1


In [ ]:
import plotly.express as px

event_scatter_map = px.scatter_map(
    df_events,
    lat="latitude",
    lon="longitude",
    color="cluster",  # Assign color by cluster ID
    hover_data={
        'start_time': True,
        'cluster': True,
        'Day_of_week': True
    },
    zoom=10,
    height=600,
    title="🗺️ Clustered Events Map"
)

event_scatter_map.update_layout(
    mapbox_style="open-street-map",
    margin={"r":0, "t":40, "l":0, "b":0}
)

event_scatter_map.show()